In [ ]:
# Actions performed on Kaggle to avoid downloading BigQuery locally

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import bq_helper
from bq_helper import BigQueryHelper

usa = bq_helper.BigQueryHelper(active_project="bigquery-public-data",dataset_name="usa_names")

In [14]:
bq_assistant = BigQueryHelper("bigquery-public-data", "usa_names")
bq_assistant.list_tables()

Using Kaggle's public dataset BigQuery integration.


['usa_1910_2013', 'usa_1910_current']

In [15]:
bq_assistant.head("usa_1910_current", num_rows=15)

,state,gender,year,name,number
0,AK,F,1910,Mary,14
1,AK,F,1910,Annie,12
2,AK,F,1910,Anna,10
3,AK,F,1910,Margaret,8
4,AK,F,1910,Helen,7
5,AK,F,1910,Elsie,6
6,AK,F,1910,Lucy,6
7,AK,F,1910,Dorothy,5
8,AK,F,1911,Mary,12
9,AK,F,1911,Margaret,7


In [16]:
bq_assistant.table_schema("usa_1910_current")

,name,type,mode,description
0,state,STRING,NULLABLE,2-digit state code
1,gender,STRING,NULLABLE,Sex (M=male or F=female)
2,year,INTEGER,NULLABLE,4-digit year of birth
3,name,STRING,NULLABLE,Given name of a person at birth
4,number,INTEGER,NULLABLE,Number of occurrences of the name


In [17]:


male_name_query = """
  SELECT
  names_step_1.name AS name,
  COALESCE(CAST(SUM(names_step_1.number) AS FLOAT64),0) AS population_amount
FROM
  `bigquery-public-data.usa_names.usa_1910_2013` AS names_step_1
WHERE
  (names_step_1.gender = 'M')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 3500;
"""
male_name_query_res = usa.query_to_pandas_safe(male_name_query)

In [18]:
female_name_query = """
  SELECT
  names_step_1.name AS name,
  COALESCE(CAST(SUM(names_step_1.number) AS FLOAT64),0) AS population_amount
FROM
  `bigquery-public-data.usa_names.usa_1910_2013` AS names_step_1
WHERE
  (names_step_1.gender = 'F')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5500;
"""
female_name_query_res = usa.query_to_pandas_safe(female_name_query)

In [30]:
combined_data = pd.concat([male_name_query_res, female_name_query_res])
unique_data = combined_data.drop_duplicates()
unique_data = unique_data.sort_values(by='population_amount', ascending=False)

print(f'Total Rows: {unique_data.shape[0]}')

Total Rows: 9000


In [31]:
unique_data.to_csv('output_names.csv')